In [1]:
RANDOM_SEED = 43

In [2]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt

z:\venvs\ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [3]:
image_size = (80,64)
batch_size = 64

In [4]:
image_transforms = transforms.Compose(
    [
        transforms.Resize(image_size), 
        transforms.ToTensor()
    ]
)

A feladat megoldását 2 külön adathalmazzal is szeretnénk megtenni:
* első és fontosabb a celeba dataset, amely celebek arcait tartalmazza előfeldolgozottan (cropped, aligned)
* második a danbooru dataset, amely anime karakterek arcait tartalmazza
* (opcionálisan egy kevert adathalmazt is szeretnénk tesztelni, hogy milyen eredményeket tudunk kapni)

Az adathalmazokat előre letöltöttük és kicsomagoltuk a tömörített fájlokat, majd így egy volume segítségével kerülnek a containerhez felcsatolásra

Mivel képgenerálásról beszélünk, a tesztelési fázis nem teljesen jelent egyértelmű feladatot
Ennek ellenére felkészülünk training, validation és test dataloaderekkel is, melyeknek bemenete a random 8:1:1 arányban felosztott adathalmaz. Kimenetük pedig egy batch_size-onként "adagolt" adathalmaz a modellünknek, image_size formájú 3 csatornás (RGB) Tensorokként

In [5]:
import paths
import os

path_list = paths.celeba
# path_list = paths.danbooru

In [9]:
generator = torch.Generator().manual_seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True

In [10]:
data = ImageFolder(root=path_list["data"], transform=image_transforms)
train_data, val_data, test_data = random_split(
    data, [0.8, 0.1, 0.1], generator=generator
)

train_dataloader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True
)
val_dataloader = DataLoader(
    val_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True
)
test_dataloader = DataLoader(
    test_data, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True
)

In [ ]:
# for x, _ in train_dataloader:
#     print(x[0].shape)
#     plt.imshow(x[0].permute(1, 2, 0))
#     break

# Model

In [ ]:
import wandb
from train import Trainer
import paths

# wandb.login()

# wandb.init(project="dl-hf")
trainer = Trainer(parallel=False)
trainer.add_paths(path_list)
trainer.add_dataloaders(train_dataloader, val_dataloader, test_dataloader)

# Training

In [ ]:
trainer.train()

# Evaluation

In [ ]:
trainer.sample(model_path=os.path.join(path_list["model"], "model.pt"))

In [6]:
base_path = path_list["data"]

In [11]:
#TODO: paths
trainer.test_FID(os.path.join(base_path, "0", "_"), os.path.join(base_path, "1", "_"))

100%|██████████| 1583/1583 [05:59<00:00,  4.40it/s]


0.3272947790833882
